In [38]:
#parameters
edf_path="C:\\Users\\bruno.grbavac\\Desktop\\Diplomski\\ \\edf_signals.edf"
hypnogram_path="C:\\Users\\bruno.grbavac\\Desktop\\Diplomski\\ \\csv_hypnogram.csv"
events_path="C:\\Users\\bruno.grbavac\\Desktop\\Diplomski\\ \\csv_events.csv"

In [39]:
# Parameters
edf_path = (
    "C:\\Users\\bruno.grbavac\\Desktop\\Diplomski\\ \\edf_signals.edf"
)

In [40]:
# ODI

In [41]:
# %matplotlib notebook
%matplotlib widget

In [42]:
import pandas as pd
import numpy as np
import csv
import mne
from scipy.signal import find_peaks
from matplotlib import pyplot as plt, dates as mdates
import seaborn as sns
import math
from datetime import datetime, timedelta, time

In [43]:
hypno_df = pd.read_csv(hypnogram_path)
events_df = pd.read_csv(events_path,sep=";")


SleepPhases = {
    'WK' : 0,
    'N1' : 1,
    'N2' : 2,
    'N3' : 3,
    'REM' : 4}
    
hypno_df['Sleep Stage'] = hypno_df['Sleep Stage'].map(SleepPhases.get)
events_df['Time'] = pd.to_datetime(events_df['Time'])
display(hypno_df)
display(events_df)

,Epoch Number,Start Time,Sleep Stage
0,1,21:36:22,0
1,2,21:36:52,0
2,3,21:37:22,0
3,4,21:37:52,0
4,5,21:38:22,0
...,...,...,...
937,938,5:24:52,0
938,939,5:25:22,0
939,940,5:25:52,0
940,941,5:26:22,0


,Type,Stage,Time,Epoch,Duration,Body Position,Validation
0,Gain (CEMG),WK,2023-06-28 21:36:22,1,"0,0",Up,Automatic Gain
1,Gain (CanFlow),WK,2023-06-28 21:36:22,1,"0,0",Up,Automatic Gain
2,Gain (TFlow),WK,2023-06-28 21:36:22,1,"0,0",Up,Automatic Gain
3,Gain (THO),WK,2023-06-28 21:36:22,1,"0,0",Up,Automatic Gain
4,Gain (ABD),WK,2023-06-28 21:36:22,1,"0,0",Up,Automatic Gain
...,...,...,...,...,...,...,...
5006,Long RR,N2,2023-06-28 05:19:20,926,"2,0",Supine,Not Validated
5007,Bradycardia,WK,2023-06-28 05:19:23,927,"27,0",Supine,Not Validated
5008,Long RR,WK,2023-06-28 05:19:31,927,"2,0",Supine,Not Validated
5009,Relative Desaturation,WK,2023-06-28 05:19:32,927,"22,0",Supine,Not Validated


In [44]:
events_df.iloc[1:2]

,Type,Stage,Time,Epoch,Duration,Body Position,Validation
1,Gain (CanFlow),WK,2023-06-28 21:36:22,1,"0,0",Up,Automatic Gain


In [45]:
events_df['Type'].unique()

array(['Gain (CEMG)', 'Gain (CanFlow)', 'Gain (TFlow)', 'Gain (THO)',
       'Gain (ABD)', 'Gain (MSnore)', 'Gain (ECG1)', 'Gain (LEG1)',
       'Gain (LEG2)', 'ok', 'Long RR', 'Bradycardia', 'Tachycardia',
       'Relative Desaturation', 'Snore', 'Obstructive Apnea', 'Arousal',
       'Hypopnea', 'Mixed Apnea', 'Central Apnea', 'Leg Movement',
       'Periodic Respiration'], dtype=object)

In [49]:
events_signal_df = pd.DataFrame(0, index=np.arange(14127500),columns=['Long RR', 'Bradycardia', 'Tachycardia','Relative Desaturation', 'Snore', 'Obstructive Apnea', 'Arousal','Hypopnea', 'Mixed Apnea', 'Central Apnea', 'Leg Movement','Periodic Respiration','Body Position'])
# d = pd.DataFrame(0, index=np.arange(len(data)), columns=feature_list)
# events_signal_df.append([0,0,0,0,0,0,0,0,0,0,0,0,0])
# events_signal_df = pd.concat([events_signal_df]*14127500 )
display(events_signal_df)
print("whole: "+str(len(events_signal_df)))

,Long RR,Bradycardia,Tachycardia,Relative Desaturation,Snore,Obstructive Apnea,Arousal,Hypopnea,Mixed Apnea,Central Apnea,Leg Movement,Periodic Respiration,Body Position
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14127495,0,0,0,0,0,0,0,0,0,0,0,0,0
14127496,0,0,0,0,0,0,0,0,0,0,0,0,0
14127497,0,0,0,0,0,0,0,0,0,0,0,0,0
14127498,0,0,0,0,0,0,0,0,0,0,0,0,0


whole: 14127500


In [50]:
def shiftTimestampToYesterday(value):
    noon = time(12, 0)
    if(noon<value.time()):
        return value - timedelta(days=1)
    else:
        return value

In [ ]:
print(datetime.now())
start_time = events_df.loc[0]['Time']
print(start_time)
noon = time(12, 0)

for index, row in events_df.iterrows():
    print("Row: " + str(index))
    secondsDiff = (shiftTimestampToYesterday(row['Time']) - shiftTimestampToYesterday(start_time)).total_seconds()
    first_row_included = int(secondsDiff*500)
    rows_duration = int(round(float(row['Duration'].replace(',','.'))*500))
    last_row_included = int(secondsDiff*500 + rows_duration)
    
    event_slice=events_signal_df.iloc[first_row_included:first_row_included+1]
    if row['Type'] in event_slice.columns:
        event_slice[row['Type']]=100
    event_slice.iloc[0]['Body Position'] = row['Body Position']

    if(rows_duration>1):
        event_slice = pd.concat([event_slice]*(rows_duration+1), ignore_index=True)
    pre_event_df_slice = events_signal_df.iloc[:first_row_included]
    post_event_df_slice = events_signal_df.iloc[last_row_included+1:]
    print("whole: "+str(len(events_signal_df))+"pre: "+ str(len(pre_event_df_slice))+", dur: "+str(rows_duration)+", event: "+str(len(event_slice)) + ", post: " + str(len(post_event_df_slice)))
    
    events_signal_df = pd.concat([pre_event_df_slice,event_slice,post_event_df_slice], ignore_index=True)
    
display(events_signal_df)
print(datetime.now())


In [ ]:
events_df[events_df['Type']=="ok"]

In [ ]:
hypno_df.rename(columns={' Epoch Number ':'Epoch','Sleep Stage': 'Stage', 'Start Time ':'Start'}, inplace=True)
hypno_df['Start'] = pd.to_datetime(hypno_df['Start'])
hypno_df['Epoch'] = hypno_df['Epoch'].astype(int)  
hypno_df['Start'] = hypno_df['Start'].apply(shiftTimestampToYesterday)
hypno_df = hypno_df.sort_values(by=['Start'])
hypno_df

In [ ]:
hypno_df.set_index('Start', inplace=True)
new_frequency = '2L'

hypno_df_resampled = hypno_df.resample(new_frequency)
hypno_df_resampled_interpolated = hypno_df_resampled.interpolate()
hypno_df_resampled_interpolated.reset_index(inplace=True)
hypno_df_resampled_interpolated['Epoch'] = hypno_df_resampled_interpolated['Epoch'].apply(round)
hypno_df_resampled_interpolated['Stage'] = hypno_df_resampled_interpolated['Stage'].apply(round)
hypno_df_resampled_interpolated

In [ ]:
fig, ax = plt.subplots()
ax.plot(hypno_df_resampled_interpolated['Start'], hypno_df_resampled_interpolated['Stage'])
ax.format_xdata = mdates.DateFormatter('%H:%M:%S')
ax.set_xlabel('Epoch Start Time')
ax.set_ylabel('Sleep Stage')
ax.set_title('Hypnogram')
plt.xticks(rotation=45)


xformatter = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(xformatter)

plt.show()

In [ ]:
hypno_df_resampled_interpolated

In [ ]:
data = mne.io.read_raw_edf(edf_path)

data_raw = data.get_data()
data_info = data.info
data_channels = data.ch_names
print(data_info)

In [ ]:
# saturation_index = data_channels.index('SpO2')
# display(saturation_index)

In [ ]:
signal_no_sleep = pd.DataFrame(data_raw)

signal_no_sleep = signal_no_sleep.T
signal_no_sleep.columns = data_channels
signal_no_sleep

In [ ]:
start_append_entry = hypno_df_resampled_interpolated.iloc[-1];
start_append_entry_start = start_append_entry['Start'] + timedelta(milliseconds=2)
missing_samples_for_edf = []

for i in range(len(signal_no_sleep)-len(hypno_df_resampled_interpolated)):
    new_entry = {'Start': start_append_entry_start + timedelta(milliseconds=2*i), 'Epoch': start_append_entry['Epoch'], 'Stage': start_append_entry['Stage']}
    missing_samples_for_edf.append(new_entry)
    
missing_samples_for_edf_df = pd.DataFrame(missing_samples_for_edf, columns=["Start", "Epoch", "Stage"])
hypno_df_resampled_interpolated = pd.concat([hypno_df_resampled_interpolated, missing_samples_for_edf_df], ignore_index=True)
print(hypno_df_resampled_interpolated)

In [ ]:
signal_to_filter = pd.concat([signal_no_sleep, hypno_df_resampled_interpolated], axis=1)
signal = signal_to_filter
print(signal)

In [ ]:
saturation = signal['SpO2']
print(saturation)

In [ ]:
x = saturation
peaks2, _ = find_peaks(x, prominence=2)      # BEST!
x[peaks2]

In [ ]:
x = saturation
fig = plt.figure()
local_minimum_indices, _ = find_peaks(-x, prominence=2)
local_maximum_indices, _ = find_peaks(x, prominence=2)
plt.plot(x)
plt.plot(local_minimum_indices, x[local_minimum_indices], 'o', label='local minimums')
plt.plot(local_maximum_indices, x[local_maximum_indices], 'x', label='local maximum')
plt.legend()
fig.canvas.layout.width = '1000px'
fig.canvas.layout.height = '500px'
plt.show()

In [ ]:
local_minimum_values = x[local_minimum_indices]
local_maximum_values = x[local_maximum_indices]
differences = []
for min_val, min_idx in zip(local_minimum_values, local_minimum_indices):
    for max_val, max_idx in zip(local_maximum_values, local_maximum_indices):
        if max_idx > min_idx:
            differences.append(max_val - min_val)


differences

In [ ]:
local_minimum_values

In [ ]:
local_maximum_values

In [ ]:
df = pd.DataFrame(differences)
df

In [ ]:
#samo one razlike veće od 3%
count = df[0][df[0] >= 3].count()
count

In [ ]:
local_minimum_values = signal.loc[local_minimum_indices]
local_maximum_values = signal.loc[local_maximum_indices]
connected_pairs = []


for i, min_idx in enumerate(local_minimum_indices):
    if(len(local_maximum_indices[local_maximum_indices < min_idx])>0):
        max_idx = local_maximum_indices[local_maximum_indices < min_idx][-1]
        connected_pairs.append((max_idx, min_idx))

differences = []
for max_idx, min_idx in connected_pairs:
    max_val = x[max_idx]
    min_val = x[min_idx]
    diff = min_idx - max_idx
    differences.append((diff, max_idx, min_idx, max_val, min_val))

df = pd.DataFrame(differences, columns=['Index Difference', 'Maximum Index', 'Minimum Index', 'Maximum Value', 'Minimum Value'])
df

In [ ]:
df2 = df

In [ ]:
df2 = df.loc[(df['Index Difference'] >= 5000) & (df['Index Difference'] <= 40000)]
df2

In [ ]:
diffDf = df2['Maximum Value'] - df2['Minimum Value']
df2 = pd.concat([df2, diffDf], axis=1)
df2.columns = [*df2.columns[:-1], 'diff']
df2

In [ ]:
df3 = df2.loc[(df2['diff'] >= 3)]

In [ ]:
zero_values = df3.loc[(df3['Minimum Value'] == 0)]

In [ ]:
df4 = df3.drop(zero_values.index)
display(df4.head(50))

In [ ]:
plt.rcParams["figure.figsize"] = [30, 20]
plt.rcParams["figure.autolayout"] = True

In [ ]:
%matplotlib

x = saturation
fig = plt.figure() # 15, 10
# plt.xticks(fontsize=50)
# plt.yticks(fontsize=50)
plt.plot(x, linewidth=1)
plt.scatter(df4['Maximum Index'], df4['Maximum Value'], color='orange', s=1)
plt.scatter(df4['Minimum Index'], df4['Minimum Value'], color='green', s=1)
plt.legend(['SpO2', 'MAX','MIN'], loc='lower right', fontsize=10)
plt.savefig('Min_MAx_find_peaks_#38011865.svg')
fig.canvas.layout.width = '1500px'
fig.canvas.layout.height = '800px'
plt.show()

In [ ]:
df2
x = saturation
fig, ax = plt.subplots(figsize=(50,20))
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
ax.plot(x, linewidth=2)
plt.scatter(df4['Maximum Index'], df4['Maximum Value'], color='orange', linewidth=4)
plt.scatter(df4['Minimum Index'], df4['Minimum Value'], color='green', linewidth=4)
for i, row in df2.iterrows():
    ax.plot([row['Maximum Index'], row['Minimum Index']], [row['Maximum Value'], row['Minimum Value']], color='purple')

plt.legend(['SpO2', 'MAX','MIN','pravac',], loc='lower right', fontsize=30)
plt.savefig('Min_MAx_find_peaks_sa_pravcem_#38011865.svg')
plt.show()

In [ ]:
saturation = saturation[saturation!= 0]
display(saturation)

In [ ]:
df4['Index Difference sek'] = df4['Index Difference']/1000
display(df4)

In [ ]:
plt.subplots(figsize=(50,20))
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.grid()
plt.xlabel('time [s]', fontsize=50)
plt.ylabel('broj uzoraka', fontsize=50)
plt.title('Vrijeme trajanja - histogram', fontsize=50)
sns.histplot(df4['Index Difference sek'], kde=True, bins=20, color='magenta')
plt.savefig('Vrijeme_trajanja_#38011865.svg')
plt.show()

In [ ]:
x = df4['Index Difference sek']

plt.subplots(figsize=(50,20))
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.grid()
plt.xlabel('time [s]', fontsize=50)
plt.ylabel('number of samples', fontsize=50)
plt.title('Duration time - histogram', fontsize=50)
plt.hist(x, density = True, bins=20, rwidth=0.9)
#sns.distplot(x, kde=True, bins=20, hist=True)
#plt.legend(['SpO2', 'MAX','MIN','pravac',], loc='lower right', fontsize=30)
plt.savefig('Duration_time_histogram_#38011865.svg')
plt.show()

In [ ]:
x = df4['diff']
plt.subplots(figsize=(50,20))
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.grid()
plt.xlabel('diff', fontsize=50)
plt.ylabel('number of samples', fontsize=50)
plt.title('diff - histogram', fontsize=50)
plt.hist(x, bins=20, rwidth=0.9, color = 'magenta')
#plt.legend(['SpO2', 'MAX','MIN','pravac',], loc='lower right', fontsize=30)
plt.savefig('diff_histogram_#38011865.svg')
plt.show()

In [ ]:
def direction_coefficient(row):
    dy = row['y2'] - row['y1']
    dx = row['x2'] - row['x1']
    return (row['y2'] - row['y1']) / (row['x2'] - row['x1'])


df = pd.DataFrame({'x1': df4['Maximum Index'],
                   'y1': df4['Maximum Value'],
                   'x2': df4['Minimum Index'],
                   'y2': df4['Minimum Value']})

df['slope'] = df.apply(direction_coefficient, axis=1)
df['angle_degrees'] = df['slope'].apply(lambda x: math.degrees(x))
df

In [ ]:
x = df['slope']
plt.subplots(figsize=(50,20))
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.grid()
plt.xlabel('slope', fontsize=50)
plt.ylabel('number of samples', fontsize=50)
plt.title('slope - histogram', fontsize=50)
plt.hist(x, bins=20, rwidth=0.9, color = 'green')
#plt.legend(['SpO2', 'MAX','MIN','pravac',], loc='lower right', fontsize=30)
plt.savefig('slope_histogram_#38011865.svg')
plt.show()